In [1]:
import os
import pandas as pd
from config import data_dir

In [3]:
df = pd.read_parquet(os.path.join(data_dir, 'all.parquet'))

label_dict = {'B007':0 , 'B021': 1, 'IR007': 2, 'IR021': 3, 'OR007@12': 4, 'OR007@3': 5, 'OR007@6': 6, 'OR021@12': 7, 'OR021@3': 8, 'OR021@6': 9}
df['label'] = df['label'].map(label_dict)

In [4]:
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split

# 取df_all的第1列到第512列作为特征
features = df.iloc[:, :512].values
# 对特征做归一化
features = (features - features.mean()) / features.std()
# 使用df_all的最后1列作为标签
labels = df.iloc[:, -1].values

# 将特征和标签转换为PyTorch张量
features = torch.tensor(features, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.long)

# 划分训练集和测试集
dataset = TensorDataset(features, labels)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [8]:
import transformers

d:\my_product\dev\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [36]:
label_dict = {v: k for k, v in label_dict.items()}
df['fault'] = df['labels'].map(label_dict)
df['fault_pred'] = df['predicted'].map(label_dict)

In [41]:
# 计算每种fault的数量，预测正确的数量
df.groupby('fault').apply(lambda x: pd.Series({
    'total': x.shape[0],
    'correct': (x['fault'] == x['fault_pred']).sum(),
    'accuracy': (x['fault'] == x['fault_pred']).mean()
}))

C:\Users\zzeee\AppData\Local\Temp\ipykernel_45120\56575370.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby('fault').apply(lambda x: pd.Series({


,total,correct,accuracy
fault,,,
B007,87.0,75.0,0.862069
B021,96.0,70.0,0.729167
IR007,97.0,92.0,0.948454
IR021,106.0,105.0,0.990566
OR007@12,87.0,46.0,0.528736
OR007@3,89.0,86.0,0.966292
OR007@6,95.0,94.0,0.989474
OR021@12,107.0,97.0,0.906542
OR021@3,94.0,87.0,0.925532
